In [3]:
import tensorflow as tf
import cv2
import numpy as np
import os
from pygame import mixer

In [4]:
# Load Haar cascades and the model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = tf.keras.models.load_model('save_models/new_best_model.h5')

In [6]:
import time  # Import for tracking time

# Initialize the alarm sound
mixer.init()
sound = mixer.Sound('sounds/alert-alarm.wav')
cap = cv2.VideoCapture(0)

score = 0
drowsy_threshold = 15
is_playing = False  # To track sound state
cool_down_timer = 0  # Time tracking variable
cool_down_duration = 5  # Cool-down duration in seconds

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    # Draw a background for the status text
    cv2.rectangle(frame, (0, height-60), (width, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), color=(255, 0, 0), thickness=2)
    
    for (ex, ey, ew, eh) in eyes:
        # Eye preprocessing
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255.0
        eye = eye.reshape(1, 80, 80, 3)

        # Predict using the model
        prediction = model.predict(eye, verbose=0)
        closed_prob = np.round(prediction[0][0], 4)
        open_prob = np.round(prediction[0][1], 4)

        # Display prediction scores
        cv2.putText(frame, f"Closed: {closed_prob}, Open: {open_prob}", (10, height-40),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=0.8, color=(255, 255, 255), thickness=1)

        # Handle drowsiness detection
        if closed_prob > 0.30:
            score += 1
            cv2.putText(frame, 'Status: DROWSY!', (10, height-20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 0, 255), thickness=2)
            if score > drowsy_threshold:
                cv2.rectangle(frame, (0, 0), (width, 50), (0, 0, 255), thickness=cv2.FILLED)
                cv2.putText(frame, 'ALERT! You are Drowsy!', (10, 30),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
                if not is_playing and time.time() - cool_down_timer > cool_down_duration:
                    sound.play(loops=-1)  # Play continuously
                    is_playing = True
        elif open_prob > 0.90:
            score -= 1
            cv2.putText(frame, 'Status: AWAKE', (10, height-20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(0, 255, 0), thickness=2)
            if score < 0:
                score = 0
            if is_playing:  # Stop sound if driver opens eyes
                sound.stop()
                is_playing = False
                cool_down_timer = time.time()  # Start the cool-down timer

    cv2.imshow('Drowsiness Detection', frame)

    # Quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [13]:
cap.release()
cv2.destroyAllWindows()